In [1]:
from test_app import data, params, model_method
from prior_comparison_tool.model import model


mod = model(model=model_method, data=data, model_kwargs=params)

In [2]:
from prior_comparison_tool.plots import compare_plot

pl = compare_plot(data={'prior1':mod})

rank     waic   p_waic d_waic weight     se dse warning waic_scale
prior1    0 -1221.34  1.79008      0      1  11.43   0   False        log
       rank     waic   p_waic d_waic weight     se dse warning waic_scale
prior1    0 -1221.34  1.79008      0      1  11.43   0   False        log
trying to plot
Launching server at http://localhost:55314
ERROR:bokeh.server.protocol_handler:error handling message
 message: Message 'PULL-DOC-REQ' content: {} 
 error: ValueError('Out of range float values are not JSON compliant')
Traceback (most recent call last):
  File "/Users/markmorrison/anaconda3/envs/msc_project/lib/python3.7/site-packages/bokeh/server/protocol_handler.py", line 90, in handle
    work = await handler(message, connection)
  File "/Users/markmorrison/anaconda3/envs/msc_project/lib/python3.7/site-packages/bokeh/server/session.py", line 67, in _needs_document_lock_wrapper
    result = func(self, *args, **kwargs)
  File "/Users/markmorrison/anaconda3/envs/msc_project/lib/python3.7

In [6]:
import panel as pn

pn.serve(pl)

Launching server at http://localhost:55135


In [12]:
import arviz as az

c = az.plot_compare(pl, show=False)


ValueError: Axis limits cannot be NaN or Inf